In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary 

## Загрузка текста

In [2]:
site = "http://az.lib.ru/k/kipling_d_r/text_0070.shtml"

chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)
#driver = webdriver.Chrome()
driver.get(site)

In [3]:
data = [i.text for i in driver.find_elements_by_xpath("//dd")]

In [4]:
data[26:29] # начало с 26 элемента

['   БРАТЬЯ МАУГЛИ',
 '  ',
 '   В Сионийских горах наступил очень жаркий вечер. Отец Волк проснулся после дневного отдыха, зевнул, почесался и одну за другой вытянул свои передние лапы, чтобы прогнать из них остаток тяжести. Волчица Мать лежала, прикрыв своей большой серой мордой четверых барахтавшихся, повизгивавших волчат, а в отверстие их пещеры светила луна.']

In [5]:
data[-3:-1] # конец на -2 элемента

['   -- Хорошо, если бы так было и в Афганистане, -- сказал старик, -- ведь там мы подчиняемся только нашей собственной воле.',
 '   -- Вот потому-то, -- заметил офицер туземцу, покручивая свой ус, -- вашему эмиру, которого вы не слушаетесь, приходится являться к нам и получать приказания от нашего вице-короля.']

In [6]:
text = ''.join(data[26:-1])

In [7]:
with open("text.txt", "w") as f:
    f.write(text)

In [ ]:
# with open("text.txt", "r") as f:
#     text = f.read()

## top 100 самых часто используемые нормализованных слов в тексте

In [8]:
#https://newtechaudit.ru/text-mining5min/

In [9]:
import pandas as pd
import numpy as np
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2

pd.set_option('display.max_rows', 100)

In [10]:
text = text.lower()

In [11]:
token = word_tokenize(text)

In [12]:
stopword = stopwords.words('russian') + [a for a in punctuation] + ['..', '--', '``', "''", '...']
token = [word for word in token if word not in stopword]

In [13]:
df = pd.DataFrame(token, columns=['original'])

In [14]:
df.head()

,original
0,братья
1,маугли
2,сионийских
3,горах
4,наступил


In [15]:
morph = pymorphy2.MorphAnalyzer()
df['normal'] = df['original'].apply(lambda x: morph.parse(x)[0].normal_form)

In [16]:
df.head()

,original,normal
0,братья,брат
1,маугли,мауголь
2,сионийских,сионийский
3,горах,гора
4,наступил,наступить


In [17]:
top100 = df['normal'].value_counts().head(100)

In [18]:
top100

сказать         312
это             232
мауголь         226
свой            224
который         200
человек         195
котик           163
маленький       148
слон            132
мочь            128
знать           120
голова          118
джунгли         118
время           117
один            117
нагой           114
мальчик         109
багира          102
волк             98
бал              95
рикки-тикки      94
говорить         92
тумая            89
нога             84
обезьяна         83
место            81
мой              80
очень            77
хан              76
шер              76
глаз             75
детёныш          73
ты               71
такой            69
кал              68
весь             67
дерево           67
стать            67
видеть           67
земля            66
наш              65
день             64
большой          64
стая             63
твой             61
акел             61
смотреть         61
морской          59
каа              58
слово            56


## все формы слова для top 10 самых часто используемые нормализованных слов в тексте

In [19]:
top10 = top100.head(10).index.tolist()

In [20]:
top10

['сказать',
 'это',
 'мауголь',
 'свой',
 'который',
 'человек',
 'котик',
 'маленький',
 'слон',
 'мочь']

In [21]:
top10_forms = df.loc[df['normal'].isin(top10)].drop_duplicates()

In [22]:
# сортировка
temp = []
for i in top10:
    temp.append(top10_forms[top10_forms['normal']==i])

top10_forms = pd.concat(temp)

In [23]:
top10_forms[['normal', 'original']]

,normal,original
40,сказать,сказал
203,сказать,сказанные
372,сказать,сказала
934,сказать,скажет
966,сказать,скажи
2199,сказать,сказать
3858,сказать,скажите
4570,сказать,сказали
6806,сказать,сказав
7927,сказать,сказано


## все имена собственные

In [24]:
#https://ru.stackoverflow.com/questions/613532/определение-имени-человека
def check_name(word):
    prob_thresh = 0.3
    for p in morph.parse(word):
        if 'Name' in p.tag and p.score >= prob_thresh:
            return word   

In [25]:
names = df['normal'].apply(check_name).dropna().unique()

In [26]:
np.sort(names).tolist()

['абиссиния',
 'агата',
 'акел',
 'алла',
 'анна',
 'арра',
 'багир',
 'багира',
 'барак',
 'бентам',
 'гапур',
 'георгий',
 'дик',
 'диков',
 'дхак',
 'кавнапур',
 'карнак',
 'карэта',
 'кеддах',
 'кханивар',
 'кханхивар',
 'лев',
 'магдаля',
 'мила',
 'надежда',
 'ната',
 'павел',
 'павлин',
 'паталамон',
 'першад',
 'пол',
 'радж',
 'ранна',
 'роза',
 'рой',
 'салаа',
 'самбхур',
 'сахиб',
 'теодор',
 'удейпур',
 'усама',
 'фернандес',
 'хая',
 'хуан',
 'худж',
 'хулла',
 'эррула']